In [2]:
import submitit
from frust.stepper import Stepper
from pathlib import Path
from tooltoad.chemutils import xyz2mol

# Run structures from folder

In [9]:
structures_path = Path("../structures/misc/")

mols = {}

for f in structures_path.rglob("*.xyz"):
    with open(f, "r") as file:
        xyz_block = file.read()
        mol = xyz2mol(xyz_block)
        mols[f.stem] = (mol, [0])

step = Stepper(list(mols.keys()), step_type="mol", save_output_dir=False)
df0 = step.build_initial_df(mols)

dfs = []
for i in range(len(df0)):
    df_tmp = df0.iloc[[i]]
    dfs.append(df_tmp)

2025-07-23 14:21:03 INFO  frust.stepper: Working dir: .


!!! Warning !!! Distance between atoms 2 and 1 (0.640000 A) is suspicious.
!!! Warning !!! Distance between atoms 13 and 12 (0.795399 A) is suspicious.
!!! Warning !!! Distance between atoms 2 and 1 (0.950001 A) is suspicious.
!!! Warning !!! Distance between atoms 3 and 1 (0.950000 A) is suspicious.


In [ ]:
def run_orca_calc(
    df,
    results_dir,
    debug=False,
    n_cores=12,
    mem_gb=30,
):
    from pathlib import Path
    name = df["custom_name"].iloc[0]

    results_dir = Path(results_dir)
    results_dir.mkdir(exist_ok=True)

    step = Stepper([name],
                step_type="none",
                debug=debug,
                save_output_dir=False,
                output_base=str(results_dir),
                n_cores=n_cores,
                memory_gb=mem_gb)

    df1 = step.orca(df, "DFT", {
        "wB97X-D3": None,
        "6-31+G**": None,
        "TightSCF": None,
        "OptTS": None,
        "Freq": None,
        "NoSym": None,
    })
    df1.to_parquet(f"{results_dir}/results_{name}.parquet")

In [ ]:
DEBUG           = False
N_CORES         = 8
MEM_GB          = 30
TIMEOUT_MIN     = 14400
RESULTS_DIR     = "results"


executor = submitit.AutoExecutor(f"logs/{RESULTS_DIR}")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

for dfi in dfs:
    name = "ts34-" + dfi["custom_name"].iloc[0] 
    executor.update_parameters(slurm_job_name=name)
    executor.submit(run_orca_calc, dfi, RESULTS_DIR, DEBUG, N_CORES, MEM_GB)
    print(f"Submitted: {name}")

Submitted: ts34-ts3
Submitted: ts34-ts4


# Run single structure

In [1]:
structures_path = Path("../structures/misc/4-trans-dimer.xyz")
mols = {}
with open(f, "r") as file:
    xyz_block = file.read()
    mol = xyz2mol(xyz_block)
    mols[f.stem] = (mol, [0])

step = Stepper(list(mols.keys()), step_type="mol", save_output_dir=False)
df0 = step.build_initial_df(mols)

NameError: name 'Path' is not defined

In [ ]:
def run_orca_calc(
    df,
    results_dir,
    debug=False,
    n_cores=12,
    mem_gb=30,
):
    from pathlib import Path
    name = df["custom_name"].iloc[0]

    results_dir = Path(results_dir)
    results_dir.mkdir(exist_ok=True)

    step = Stepper([name],
                step_type="none",
                debug=debug,
                save_output_dir=False,
                output_base=str(results_dir),
                n_cores=n_cores,
                memory_gb=mem_gb)

    df1 = step.orca(df, "DFT", {
        "wB97X-D3": None,
        "6-31G**": None,
        "TightSCF": None,
        "SlowConv": None,
        "Opt":     None,
        "Freq":    None,
        "NoSym":   None,
    })
    
    df2 = step.orca(df1, "DFT-SP", {
        "wB97X-D3": None,
        "6-31+G**": None,
        "TightSCF": None,
        "SP":       None,
        "NoSym":    None,
    })

    df2.to_parquet(f"{results_dir}/results_{name}.parquet")

In [ ]:
DEBUG           = False
N_CORES         = 10
MEM_GB          = 30
TIMEOUT_MIN     = 14400
RESULTS_DIR     = "4-trans-dimer"
OPTIONS={
    "wB97X-D3": None,
    "6-31G**": None,
    "TightSCF": None,
    "Opt" : None,
    "Freq": None,
    "NoSym": None,
}

executor = submitit.AutoExecutor(f"logs/{RESULTS_DIR}")
executor.update_parameters(
    slurm_partition="kemi1",
    cpus_per_task=N_CORES,
    mem_gb=MEM_GB,
    timeout_min=TIMEOUT_MIN,
)

name = df0["custom_name"].iloc[0]
executor.update_parameters(slurm_job_name=name)
executor.submit(run_orca_calc, df0, RESULTS_DIR, DEBUG, N_CORES, MEM_GB)
print(f"Submitted: {name}")

Submitted: 4-trans-dimeropt
